# Enunciado Desafio Codenation 06/2020
Detalhes

O contexto do desafio gira em torno dos resultados do ENEM 2016 (disponíveis no arquivo train.csv). Este arquivo, e apenas ele, deve ser utilizado para todos os desafios. Qualquer dúvida a respeito das colunas, consulte o Dicionário dos Microdados do Enem 2016.


Muitas universidades brasileiras utilizam o ENEM para selecionar seus futuros alunos e alunas. Isto é feito com uma média ponderada das notas das provas de matemática, ciências da natureza, linguagens e códigos, ciências humanas e redação, com os pesos abaixo:

> matemática: 3

> ciências da natureza: 2

> linguagens e códigos: 1.5

> ciências humanas: 1

> redação: 3

No arquivo test.csv crie um modelo para prever nota da prova de matemática (coluna NU_NOTA_MT) de quem participou do ENEM 2016.

Salve sua resposta em um arquivo chamado answer.csv com duas colunas: NU_INSCRICAO e NU_NOTA_MT.

Faça o upload do arquivo answer.csv usando o botão “Submeter resposta”.

In [2]:
#importando as bibliotecas
import pandas as pd  #bibioteca responsável para o tratamento e limpeza dos dados
import numpy as np #biblioteca utilizada para o tratamento eficiente de dados numéricos
import datetime  #biblioteca utilizada para trabalhar com datas
from matplotlib import pyplot as plt  #plotar os gráficos
import seaborn as sns #plot de gráficos
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
#from google.colab import drive

from datetime import datetime # para calcular o tempo de execução do modelo
import time # para calcular o tempo de execução do modelo

from sklearn.metrics import r2_score #método para o cálculo do R2
from sklearn.preprocessing import StandardScaler, MinMaxScaler # utilizado para o preprocessamento antes de entrar com os modelos
from sklearn.pipeline import Pipeline # pipeline para facilitar a aplicação dos modelos

#modelos que utilizamos para os testes
from sklearn import linear_model
from sklearn import svm
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor

In [3]:
# monta o Drive para exportar o arquivo final
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [3]:
# Carrega datases 
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [5]:
train

,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,...,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,Q028,Q029,Q030,Q031,Q032,Q033,Q034,Q035,Q036,Q037,Q038,Q039,Q040,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,1,4314902.0,Porto Alegre,43.0,RS,1,4,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,A,A,A,A,A,B,A,A,C,A,A,A,C,H,E,5.0,5.0,1.0,0.0,0.0,5,5,0,5,5,5,5,5.0,A,A,A,A,A,A,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,3,1,2304707.0,Granja,23.0,CE,2,0,2,1.0,0,23005157.0,2304707.0,Granja,23.0,CE,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,A,A,A,A,A,B,A,A,A,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,1,1,1,1,1,NaN,A,A,C,A,B,A,A,C,A
2,3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,3,1,2304400.0,Fortaleza,23.0,CE,3,0,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,5,5,5,5,5,NaN,A,A,A,A,C,A,A,B,A
3,4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,0,1,3304557.0,Rio de Janeiro,33.0,RJ,1,9,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,B,B,A,B,A,B,C,A,B,D,B,C,B,C,F,C,2.0,0.0,5.0,5.0,5.0,5,5,5,5,2,5,5,5.0,C,A,A,A,A,D,A,A,A
4,5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,2,1,1505304.0,Oriximiná,15.0,PA,1,4,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,B,A,A,A,A,B,A,A,B,A,A,A,B,F,D,5.0,4.0,3.0,5.0,4.0,5,5,4,3,1,4,5,NaN,A,A,A,A,A,A,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,4582,aadf671ef8c1c6efa42e69432093ab83c17a52ac,2016,3502903,Araçoiaba da Serra,35,SP,20,M,0.0,1,1,3552205.0,Sorocaba,35.0,SP,1,3,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,B,A,B,A,A,C,B,B,E,B,B,B,B,C,A,5.0,5.0,5.0,5.0,5.0,5,5,5,5,5,5,5,NaN,A,A,A,A,A,A,A,A,A
13726,4583,461f7ef2753d9d5fa0b054930d6b0e0eec318e81,2016,2613800,São Vicente Ferrer,26,PE,33,F,0.0,3,2,NaN,NaN,NaN,NaN,1,10,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,A,A,A,A,A,B,A,A,B,B,A,B,B,I,A,4.0,4.0,5.0,5.0,4.0,5,5,5,5,5,5,5,NaN,A,A,B,B,A,A,A,A,A
13727,4584,5354503700ecf2388f68990435390b6e1ad454ee,2016,4322186,Tupanci do Sul,43,RS,15,F,0.0,1,1,4303202.0,Cacique Doble,43.0,RS,3,0,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,B,A,A,A,A,B,A,A,B,A,A,B,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,3,2,2,2,2,2,NaN,A,A,A,A,C,A,A,A,A
13728,4585,c7a9e94a42bd76b7978f13c013a61702e36cc42c,2016,3303203,Nilópolis,33,RJ,36,M,1.0,3,1,3304557.0,Rio de Janeiro,33.0,RJ,4,0,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,B,A,A,B,A,B,A,A,C,B,B,B,C,E,E,5.0,5.0,5.0,5.0,5.0,5,5,2,5,5,3,5,5.0,A,A,A,A,D,A,A,A,D


In [6]:
test.describe()

,CO_UF_RESIDENCIA,NU_IDADE,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,TP_DEPENDENCIA_ADM_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DISLEXIA,IN_DISCALCULIA,IN_SABATISTA,IN_GESTANTE,IN_IDOSO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO
count,4576.000000,4576.000000,4576.000000,4576.000000,4576.000000,4576.000000,4576.000000,1480.000000,4576.000000,1480.000000,4576.000000,4576.0,4576.000000,4576.0,4576.0,4576.000000,4576.000000,4576.0,4576.000000,4576.000000,4576.000000,3442.000000,3442.000000,3377.000000,4576.000000,3377.000000,3377.000000,3377.000000,3377.000000,3377.000000,3377.000000,3377.000000
mean,30.762019,21.542395,2.181163,1.060970,1.791084,2.119537,1.368663,1.182432,0.127404,2.264865,0.000874,0.0,0.000219,0.0,0.0,0.010271,0.000437,0.0,0.752185,0.752185,0.739729,473.136287,529.195264,514.304205,0.604458,1.095943,119.626888,111.915902,106.384365,112.881256,76.683447,527.491857
std,9.912313,6.854073,1.017531,0.281289,0.875692,3.210353,0.568589,0.572495,0.333461,0.716922,0.029556,0.0,0.014783,0.0,0.0,0.100835,0.020904,0.0,0.431791,0.431791,0.440818,70.995131,73.086665,67.725564,0.489020,0.695047,29.028146,33.296849,33.834492,33.202244,42.985637,151.979490
min,11.000000,14.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,17.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,2.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.000000,1.000000,0.000000,419.625000,479.725000,468.400000,0.000000,1.000000,120.000000,100.000000,80.000000,100.000000,40.000000,440.000000
50%,31.000000,19.000000,3.000000,1.000000,2.000000,0.000000,1.000000,1.000000,0.000000,2.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.000000,1.000000,1.000000,458.250000,532.000000,516.800000,1.000000,1.000000,120.000000,120.000000,100.000000,120.000000,80.000000,540.000000
75%,35.000000,23.000000,3.000000,1.000000,2.000000,3.000000,2.000000,1.000000,0.000000,2.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.000000,1.000000,1.000000,514.350000,579.550000,562.400000,1.000000,1.000000,140.000000,120.000000,120.000000,120.000000,100.000000,600.000000
max,53.000000,65.000000,5.000000,4.000000,4.000000,10.000000,3.000000,3.000000,1.000000,4.000000,1.000000,0.0,1.000000,0.0,0.0,1.000000,1.000000,0.0,1.000000,1.000000,2.000000,778.900000,773.600000,708.900000,1.000000,9.000000,200.000000,200.000000,200.000000,200.000000,200.000000,980.000000


In [7]:
# pega as colunas com correlação maior que 0,3

corr = train.corr()[train.corr()['NU_NOTA_MT'] > 0.3]['NU_NOTA_MT']
corr_features = corr.index.to_list()
corr_features.remove('NU_NOTA_MT')
corr_features

['TP_DEPENDENCIA_ADM_ESC',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'NU_NOTA_REDACAO']

In [0]:
# vamos usar as variaveis train_inscricao_notas e test_inscricao_notas
# são variaveis apenas com as features corr_features

train_inscricao_notas = train[corr_features]
test_inscricao_notas = test[corr_features]

In [0]:
# tentamos substituir os valores nulos das colunas pela média mas os resultados não foram satisfatorios.
# então, substitui por 0

train_inscricao_notas = train_inscricao_notas.copy()
test_inscricao_notas = test_inscricao_notas.copy()

for feature in corr_features:
  train_inscricao_notas[feature].fillna(0, inplace=True) #substituindo os valores nulos por 0
  test_inscricao_notas[feature].fillna(0, inplace=True) #substituindo os valores nulos por 0
  # train_inscricao_notas[feature].fillna(train_inscricao_notas[feature].mean(), inplace=True) #substituindo os valores nulos pela média da coluna
  # test_inscricao_notas[feature].fillna(test_inscricao_notas[feature].mean(), inplace=True) #substituindo os valores nulos pela média da coluna

# puxa a feature NU_NOTA_MT para utilizar como saida do modelo
train_inscricao_notas['NU_NOTA_MT'] = train['NU_NOTA_MT']
train_inscricao_notas['NU_NOTA_MT'].fillna(0, inplace=True) #substituindo os valores nulos por 0
# train_inscricao_notas['NU_NOTA_MT'].fillna(train['NU_NOTA_MT'].mean(), inplace=True)

In [10]:
# verifica se ficou algum nulo
train_inscricao_notas.isnull().sum()

TP_DEPENDENCIA_ADM_ESC    0
NU_NOTA_CN                0
NU_NOTA_CH                0
NU_NOTA_LC                0
NU_NOTA_COMP2             0
NU_NOTA_COMP3             0
NU_NOTA_COMP4             0
NU_NOTA_COMP5             0
NU_NOTA_REDACAO           0
NU_NOTA_MT                0
dtype: int64

In [0]:
#separa em treino e teste

x = train_inscricao_notas.astype(int)
y = train_inscricao_notas['NU_NOTA_MT'].astype(int)
x_train, x_test, y_train, y_test = train_test_split(x[corr_features], y, test_size=0.05, shuffle=True, random_state=0)

In [0]:
#CRIANDO UM PIPELINE PARA FAZER O vectorizer => transformer => classifier
pipes = {  
  'KNeighborsRegressor': Pipeline([
      ('clr', MinMaxScaler()),
      ('clf', KNeighborsRegressor(n_neighbors=15)),
  ]),
  'KernelRidge': Pipeline([
      ('clr', MinMaxScaler()),
      ('clf', KernelRidge(alpha=1.0)),
  ]),
  'LinearRegression': Pipeline([
      ('clr', MinMaxScaler()),
      ('clf', LinearRegression()),
  ]),
  'PassiveAggressiveRegressor': Pipeline([
      ('clr', MinMaxScaler()),
      ('clf', PassiveAggressiveRegressor()),
  ]),
  
  'Ridge': Pipeline([
      ('clr', StandardScaler()),
      ('clf', linear_model.Ridge(alpha=.5)),
  ]),

  'SVR': Pipeline([
      ('clr', StandardScaler()),
      ('clf', svm.SVR()),
  ]),

  'GradientBoostingRegressor': Pipeline([
      ('clr', StandardScaler()),
      ('clf', GradientBoostingRegressor()),
  ]),

  'DecisionTreeRegressor': Pipeline([
      ('clr', StandardScaler()),
      ('clf', tree.DecisionTreeRegressor()),
  ])
}

In [16]:
# roda os modelos e pega o que tiver melhor R2
melhor_resultado = 0

for nome, modelo in pipes.items():
    start_time = time.time()
    modelo.fit(x_train, y_train)
    predicted = modelo.predict(x_test)
    R_2 = r2_score(y_test, predicted)  #realiza o cálculo do R2
    print(nome + "=> (R2):"+ str(round(R_2,2)))

    if melhor_resultado < R_2:
      modelo_resultado = {}
      modelo_nome, modelo_treinado, modelo_resultado = nome, modelo, R_2
      melhor_resultado = R_2

KNeighborsRegressor=> (R2):0.91
KernelRidge=> (R2):0.9
LinearRegression=> (R2):0.9
PassiveAggressiveRegressor=> (R2):0.9
Ridge=> (R2):0.9
SVR=> (R2):0.88
GradientBoostingRegressor=> (R2):0.92
DecisionTreeRegressor=> (R2):0.84


In [18]:
# Modelo com melhor resultado
print (modelo_nome + ': com R2= '+ str(modelo_resultado))

GradientBoostingRegressor: com R2= 0.9195522389473596


In [0]:
# roda melhor modelo na base de teste
test_inscricao_notas['NU_NOTA_MT'] = modelo_treinado.predict(test_inscricao_notas[corr_features].astype(int))

In [20]:
#cria df apenas com os resultados que precisam ser exportados
result = pd.DataFrame({'NU_INSCRICAO': test['NU_INSCRICAO'], 'NU_NOTA_MT': test_inscricao_notas['NU_NOTA_MT']})
result

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,434.032348
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,454.029324
2,b38a03232f43b11c9d0788abaf060f7366053b6d,589.769923
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,0.426361
4,715494628a50142ce8cb17191cfe6d0f3cae0934,537.780278
...,...,...
4571,dac0f22429c7f8e3931d0abaf5dfc8e5c772a48b,447.209120
4572,a75fa8770257e7c9368d059fe53d9ef431f4bdef,469.959204
4573,655fa6306720ff16e825903b5422a46608a77545,660.101134
4574,1f4bc3e3d56212d500625bfe8ac78ccff4362293,448.920180


In [0]:
# Exporta
now = datetime.now()
dt_string = now.strftime("%H-%M-%S")

result.to_csv('/drive/My Drive/'+'answer_'+dt_string+'.csv', index=False)